# Python for Data Analytics - Exercises 3 #

# Computing Expectations by using Numerical Integration #

## Objectives ##

* To gain experience with some more advanced Python programming features: passing functions as parameters; defining 'anonymous’ functions.
* To see how numerical integration can be used to approximate the [mean](https://en.wikipedia.org/wiki/Mean) and [variance](https://en.wikipedia.org/wiki/Variance) of a distribution.

## 1. Background ##

Expectations of continuous random variables (e.g. means and variances) can be computed from the probability density function by integration. ‘Analytic’ integration can be used to derive exact expressions for the mean and variance of the Gaussian distribution and the uniform distribution. Sometimes integration can be hard to perform analytically. In such cases it is possible to resort to approximate numerical techniques to compute the mean and variance. There are two main approaches that can take:
<ol>
    <li>[monte-carlo sampling](https://en.wikipedia.org/wiki/Monte_Carlo_method): (i.e. generating samples from the distribution and computing their statistics directly)</li>
    <li>integration by [numerical quadrature](https://en.wikipedia.org/wiki/Numerical_integration), (i.e. approximating the area under a curve by cutting it into a finite number of slithers that we then sum up).</li>
</ol>

Quadrature works very well for 1-D problem and this will be the focus of this notebook.

# PART 1 – Numerical quadrature #

## 2. Introduction ##

Numerical quadrature is a form of numerical integration and can be used to integrate functions between definite limits, i.e. finding the area under a curve between a lower and upper limit on the x-axis. There are several different versions of the technique but they all work in the same way. The area under the curve is approximated by summing a series of segments whose area is easy to compute. In the simplest case the segments are very thin rectangles and the area under the curve can be approximated by summing the area of each of the rectangles shown. Note that the width of each rectangle is some fixed step size, and the heights are calculated by evaluating the function f(x) at the x position where the rectangle is centered. The area of each is simply the width times the height.

The more rectangles we use, the thinner they will be and the closer the sum of their areas will be to the true area under the curve. i.e. by summing more rectangles we can improve the precision of the approximation – at the expense of additional computational effort.

## 3. Passing function as arguments ##

Python already has functions for performing numeric integration but in order to get some more Python programing experience you are going to write your own. Your function needs to take four inputs:

* f() - the function we wish to integrate,
* a - the lower bound,
* b - the upper bound and
* N - the number of segments to be used to approximate the integral.

Note, the 1st parameter is a function – we are passing a function to a function. In Python it is possible to pass functions directly. Let’s illustrate this by passing the math.cos() function as a parameter to a function called squared that can compute the squared output of any function:
First write the function squared like this

    function y=squared(fn, x)
    y = fn(x);
    y=y.^2;

In [ ]:
def squared(fn, x):
    y = fn(x)
    y = y * y
    return y

Now to compute the square of cos(pi/3) we could call our function as,

In [ ]:
import numpy as np    
squared(np.cos, np.pi/3.0)

Now if we wanted to compute the square of sin(pi/2) we could simply write,

In [ ]:
squared(np.sin, np.pi/3.0)

What happens if you try the following,

    squared(np.sin, np.linspace(0,2*np.pi,100))
    
Try it in the cell below,

The above example works because np.sin() has been designed to be able to take a vector of inputs and compute the sin of every element in the vector. All the numpy functions will work in this way. This makes the numpy library very fast and powerful.

## 4. Numerical integration ##

We now need to write our numerical integration function.

Say we want to integrate $f()$ between $a$ and $b$. We are first going to evaluate $f(x)$ at lots of
positions between $a$ and $b$, (say $N$ positions).

To do this we can first generate a number line with $N$ points between a and b stored in the vector
x. We can do this using numpy's [linspace](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) function x = np.linspace(a, b, N)

We then pass the vector $x$ to our function $f()$ in the usual way.

    fx = f(x)

This gives us the heights of the rectangles on the previous page. There will be $N$ rectangles between $a$ and $b$ so we can work out that their width will be $(b-a)/N$. So the area of each one is $fx(i) . (b-a)/N$. So our integral, which is the total area of all the rectangles combined, is given by

    np.sum(fx)*(b-a)/N

The more slithers we use the better the accuracy will be.
The function signature for our integrate function should look like

    integrate(f, a, b, N)
    
Putting this all together we have the code below

In [ ]:
def integrate(f, a, b, N):
    x = np.linspace(a, b, N)
    fx = f(x)
    area = np.sum(fx)*(b-a)/N
    return area

We can now use the function to integrate a sine curve between 0 and and pi/2. This should produce 1. Let's run it using 100 points

In [ ]:
integrate(np.sin, 0, np.pi/2, 100)

The answer is off by about 0.002. This is not bad. In fact the integrate function above is simple but it is not *quite* right. If you consider the points generated by linspace then you might be able to spot the error. Try fixing it in rerunning the test. If you get it right the error will be about 200 times smaller (Rewrite the integrate function in the cell below)

## 5. Computing the mean of a distribution ##

Remember, if x is a random variable with distribution p(x) then its mean value, E(x) is given by integrating x.p(x) between plus and minus infinity.

We will now make a new function called compute_mean that takes the distribution as an input function. The compute_mean will also take the upper and lower bounds and N slices parameters that will be passed on to the integrate function. So define a new function with the following signature,

    function m = compute_mean(f, a, b, N)

Inside the body of this function we will use our function integrate to evaluate the integral of x.f(x).

But how can we pass x.f(x) to integrate? integrate() takes the function that we want to integrate as input, but the problem is that we haven’t written a function that directly evaluates x.f(x).

The solution is to use a Python inner function to define a new function inside the compute_mean function body.


`compute_mean` has been passed a function called f and we want to write a new function that evaluates x.f(x). Elementwise multiplication of two vectors can be done using numpy's multiply() function. So our inner-function will look like 

    def xfx(x):
        return np.multiply(f(x), x)
        
Note that f2() did not need to be passed the function f() as it inherited from the scope of the containing function compute_mean()    

Once this function has been defined we can now integrate it using our integrate function and so compute the mean, i.e.

In [ ]:
def compute_mean(f, a, b, N):
    def xfx(x):
        return np.multiply(x, f(x))
    mean = integrate(xfx, a, b, N)
    return mean

compute_mean(np.cos, 0, np.pi/2, 10)

## 6. Testing the compute_mean function ##

We can now test our compute mean function. The Python scipy.stats package has functions to handle many distributions. To get a function that evaluates the [pdf](https://en.wikipedia.org/wiki/Probability_density_function) of a [Gaussian](https://en.wikipedia.org/wiki/Normal_distribution) we first generate a random variable using

     x = norm(0, 1)

and then we can call

    x.pdf()

In [ ]:
from scipy.stats import norm
x = norm(0, 1)
compute_mean(x.pdf, -5, 5, 100)

Experiment with different a, b and N parameters below and compare the results. 

To evaluate a normal pdf with mean 5 and variance 4 we simply need to call the norm function with different mean and standard deviation parameters

    x = norm(5, 2)
    
Experiment with different values of the mean and check that the mean estimation still works.

In [ ]:
# Estimate mean of normal with mean = 5, var = 4
print (compute_mean(norm(5,2).pdf, -10, 10, 1000))
# Estimate mean of normal with mean = 2, var = 9
print (compute_mean(norm(2,3).pdf, -10, 10, 1000))
# Estimate mean of normal with mean = -2, var = 1
print (compute_mean(norm(-2,1).pdf, -10, 10, 1000))

(Note, the 3rd parameter sets the standard deviation – the square root of variance – so for a variance of 4 we need to set the standard deviation to 2).

We can compute the mean of the uniform pdf using similar code,

    from scipy.stats import uniform
    x = uniform(0, 1)
    
The two parameters are now the range of the uniform variable. i.e. x is uniform between 0 and 1.

Try estimating the mean for various different uniform distributions.

## 7. Computing the variance ##

We will now extend compute_mean so that it also computes the variance. Make a new function called `compute_mean_and_variance` with the same parameters as `compute_mean` but which returns both a mean and a variance

    compute_mean_and_variance(f, a, b, N) 

Copy the code from `compute_mean` so that the mean computation is complete.

Variance is E((x-E(x))^2). This can be rearranged into a handier form as var(x) = E(x^2) – E(x)^2.

Our function already computes E(x) i.e. the mean. We now just need to extend it to compute E(x^2). i.e. we need to integrate x^2.p(x) . Do this using the same steps described in section 3. i.e. you will need to define another anonymous function in the `compute_mean_and_variance` function body to evaluate x^2.p(x) and then pass this to the integrate function to compute E(x^2). Once you have evaluated E(x) and E(x^2) you can use them to compute var(x).

Write your code in the cell below.

## 8. Evaluating the estimates ##

Generate a normal random variable with mean 5 and variance 4  (i.e. x = norm(5,2))

Set the limits of the integration to be -100 to 100. Now call `compute_mean_and_variance`
with increasing values of N.

Given that in this case we know the true mean and variance (i.e. 5 and 4) we can compute the
error in the estimates (i.e. the difference between the estimates and the true values). 

Make a plot of the estimation error as a function of N.

How quickly does the error decrease with increasing N?

## 9. Challenge: Using your code ##

A variable x has a pdf defined by $p(x)=30*x*(1-x)^4$ for $x>=0$ and $x<=1$
and $p(x) = 0$ for $x<=0$ and $x>=1$.

Use your code to compute the mean and variance of this distribution.

# PART 2 – Programming Challenge #

Bayesian inference assumes that the probability of each attribute belonging to a given class value is independent of all other attributes. Conditional probability is the probability of a class value given a value of an attribute; by multiplying the conditional probabilities together for each attribute for a given class value, we have a probability of a data instance belonging to that class. This is also represented by the frequently seen formula

$$P(A|B) = \frac{P(B | A) P(A)}{P(B)}$$

where

* P(A|B) is the posterior probability of class (A) given predictor (B)
* P(A) is the prior probability of class
* P(B|A) is the likelihood
* P(B) is the prior probability of predictor

## 10. Programming exercise ##

#### Exercise 1. #### 

Simulate tossing a fair coin 1,000,000 times. Count the length of each sequence of identical outcomes. Plot a histogram of the result.

For example, the following sequence, H H H T T H H H H T T T, would count as 1 sequence of 2 (T T); 2 sequences of 3 (H H H and T T T) and 1 sequence of 4 (H H H H).

Hint: the coin toss can be simulated by using the function `rand’ to pick a number between 0 and 1 and outputting heads if the number is greater than 0.5 or else outputting tails.
Write your solution using a for-loop to start with.

Can you find a way of `vectorising’ your algorithm – i.e. writing it without loops? Is it faster?
    
Time your solutions using timeit. 

#### Exercise 2. ####

Bias the coin so that it comes up heads with a probability, p, greater than 0.5. How does the shape of the histogram vary with increasing p?